In [1]:
import os
os.chdir("D:/Avinash/applied_AI/Module 3-NLP/assignment")
os.getcwd()

'D:\\Avinash\\applied_AI\\Module 3-NLP\\assignment'

In [2]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [3]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [4]:
# it accepts only list of sentances
def fit(dataset): 
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        return vocab
    else:
        print("you need to pass list of sentance")

In [5]:
type(fit(corpus))

dict

In [6]:
words=fit(corpus)

In [7]:
vocab=list(words.keys())

In [8]:
print(vocab)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [9]:
def IDF(dataset,unique_words):
    idf_dict={}
    N=len(dataset)
    #words=set()
    for i in unique_words:
        count=0
        for row in dataset:
            for word in row.split(" "):
                if len(word) < 2:
                    continue
                if (i == word):
                    count+=1
                    break
        idf_dict[i]=1+math.log((1+N)/(1+count))
    return idf_dict      

In [10]:
idf_dict=IDF(corpus,vocab)

In [11]:
idf_dict

{'and': 1.916290731874155,
 'document': 1.2231435513142097,
 'first': 1.5108256237659907,
 'is': 1.0,
 'one': 1.916290731874155,
 'second': 1.916290731874155,
 'the': 1.0,
 'third': 1.916290731874155,
 'this': 1.0}

In [12]:
print(list(IDF(corpus,vocab).values()))

[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]


In [13]:
print(words)

{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}


In [14]:
#tfidf=[]
def transform_1(dataset,vocab,idf_dict):
    rows = []
    columns = []
    values = []
    for id,row in enumerate(tqdm(dataset)):
        for word in row.split():
            word_freq=dict(Counter(row.split()))
            for w,freq in word_freq.items():
                if w in list(vocab.keys()):
                    if len(w) < 2:
                        continue
                    tf=freq/len(row.split())
                    tfidf=(tf*idf_dict[w])
                    col_index = vocab.get(w)
                    rows.append(id)
                    columns.append(col_index)
                    values.append(tfidf)
    sparse_matrix=csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    #print("NORM FORM",normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False))
    output =normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)
    return output    

In [15]:
output_1=transform_1(corpus,words,idf_dict)

100%|██████████| 4/4 [00:00<00:00, 2000.62it/s]


In [16]:
output_1.shape

(4, 9)

In [17]:
output_1[0]

<1x9 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [18]:
print(output_1[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


Task-2

In [19]:
import pickle
with open('cleaned_strings', 'rb') as f:
    corpus_1 = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus_1))

Number of documents in corpus =  746


In [20]:
def transform_fit_idf(dataset): 
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        #print("Unique words :",list(vocab))

#code for calculating the idf values and sorting of the vocabulary
    idf_dict={}
    N=len(dataset)
    #words=set()
    for i in unique_words:
        count=0
        for row in dataset:
            for word in row.split(" "):
                if len(word) < 2:
                    continue
                if (i == word):
                    count+=1
                    break
        idf_dict[i]=1+math.log((1+N)/(1+count))
    idf_sorted={key:value for (key, value) in sorted(idf_dict.items() ,  key=lambda x: x[1] ,reverse=True ) }
    idf_1= dict(list(idf_sorted.items())[0:50]) 
    vocab_sorted=idf_1.keys()
    dict2 = {x:i for i, x in enumerate(vocab_sorted)}
    print("sorted vocubulary based on top 50 idf values: ",dict2)
    print("------------------------------------------------------------------------------------------------------------")
    print("top 50 idf values :",idf_1)

#code for generating sparse matrix
    rows = []
    columns = []
    values = []
    for id,row in enumerate(tqdm(dataset)):
        for word in row.split():
            word_freq=dict(Counter(row.split()))
            for key in dict2.keys():
                for w,freq in word_freq.items():
                    if (key==w):
                        tf=freq/len(row.split())
                        tfidf=(tf*idf_1[key])
                        col_index = dict2.get(key)
                        rows.append(id)
                        columns.append(col_index)
                        values.append(tfidf)
    sparse_matrix=csr_matrix((values, (rows,columns)), shape=(len(dataset),len(dict2)))
    print("NORM FORM",normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False))
    output =normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)
    return output
   


In [21]:
output_4=transform_fit_idf(corpus_1)

  0%|          | 0/746 [00:00<?, ?it/s]

sorted vocubulary based on top 50 idf values:  {'aailiyah': 0, 'abandoned': 1, 'abroad': 2, 'abstruse': 3, 'academy': 4, 'accents': 5, 'accessible': 6, 'acclaimed': 7, 'accolades': 8, 'accurate': 9, 'accurately': 10, 'achille': 11, 'ackerman': 12, 'actions': 13, 'adams': 14, 'add': 15, 'added': 16, 'admins': 17, 'admiration': 18, 'admitted': 19, 'adrift': 20, 'adventure': 21, 'aesthetically': 22, 'affected': 23, 'affleck': 24, 'afternoon': 25, 'aged': 26, 'ages': 27, 'agree': 28, 'agreed': 29, 'aimless': 30, 'aired': 31, 'akasha': 32, 'akin': 33, 'alert': 34, 'alike': 35, 'allison': 36, 'allow': 37, 'allowing': 38, 'alongside': 39, 'amateurish': 40, 'amaze': 41, 'amazed': 42, 'amazingly': 43, 'amusing': 44, 'amust': 45, 'anatomist': 46, 'angel': 47, 'angela': 48, 'angelina': 49}
------------------------------------------------------------------------------------------------------------
top 50 idf values : {'aailiyah': 6.922918004572872, 'abandoned': 6.922918004572872, 'abroad': 6.92291

100%|██████████| 746/746 [00:04<00:00, 186.00it/s]

NORM FORM   (0, 30)	1.0
  (68, 24)	1.0
  (72, 29)	1.0
  (74, 31)	1.0
  (119, 33)	1.0
  (135, 3)	0.3779644730092272
  (135, 10)	0.3779644730092272
  (135, 18)	0.3779644730092272
  (135, 20)	0.3779644730092272
  (135, 36)	0.3779644730092272
  (135, 40)	0.3779644730092272
  (135, 41)	0.3779644730092272
  (176, 49)	1.0
  (181, 13)	1.0
  (192, 21)	1.0
  (193, 23)	1.0
  (216, 2)	1.0
  (222, 47)	1.0
  (225, 19)	1.0
  (227, 17)	1.0
  (241, 44)	1.0
  (270, 1)	1.0
  (290, 25)	1.0
  (333, 26)	1.0
  (334, 15)	1.0
  (341, 43)	1.0
  (344, 42)	1.0
  (348, 8)	1.0
  (377, 37)	1.0
  (409, 5)	1.0
  (430, 39)	1.0
  (457, 45)	1.0
  (461, 4)	1.0
  (465, 38)	1.0
  (475, 35)	1.0
  (493, 6)	1.0
  (500, 48)	1.0
  (548, 0)	0.7071067811865476
  (548, 32)	0.7071067811865476
  (608, 14)	1.0
  (612, 11)	1.0
  (620, 46)	1.0
  (632, 7)	1.0
  (644, 12)	0.7071067811865475
  (644, 27)	0.7071067811865475
  (664, 28)	1.0
  (667, 22)	1.0
  (691, 34)	1.0
  (697, 9)	1.0
  (722, 16)	1.0


In [22]:
output_4.shape

(746, 50)

In [23]:
output_4[0].shape

(1, 50)

In [24]:
output_4[0].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])